# Bibliothèques

In [142]:
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
from hubspot import HubSpot


# Connection à l'API

In [143]:
token="pat-eu1-52e0d25c-7a77-4128-bf28-4787e96528bf"
api_client = HubSpot(access_token=token)

# Calcul du montant total des deals sans pondération

In [144]:

def total_deal_value():
    
    # Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
    deals_id=[]
    deals=api_client.crm.deals.get_all()
    for deal in deals:
        deals_id.append(deal.to_dict()["id"])

    # Calcul du montant
    montant_deals=0
    for id in deals_id:
        deal_data=api_client.crm.deals.basic_api.get_by_id(id,properties=["amount"]).to_dict()
        montant_deals+=float(deal_data["properties"]["amount"])
    return montant_deals

In [146]:
# Résulat
total_deal_value()

1665098.0

# Calcul du montant total des deals avec pondération

In [124]:
def total_deal_value_weighted():

    # Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
    deals_id=[]
    deals=api_client.crm.deals.get_all()
    for deal in deals:
        deals_id.append(deal.to_dict()["id"])
    
    # Calcul du montant total des deals avec pondération
    montant_deals_pond=0
    for id in deals_id:
        deal_data=api_client.crm.deals.basic_api.get_by_id(id,properties=["amount","hs_deal_stage_probability"]).to_dict()
        montant_deals_pond+=float(deal_data["properties"]["amount"]) * float(deal_data["properties"]["hs_deal_stage_probability"])
    return montant_deals_pond


In [147]:
# Résultat
total_deal_value_weighted()

513588.2

### Update du montant d'un deal


In [148]:
def update_deal(deal_id,amount):
    try:
        update_montant_deal = SimplePublicObjectInput(
                    properties={"amount":amount})
        api_client.crm.deals.basic_api.update(simple_public_object_input=update_montant_deal,deal_id=deal_id)
        return print(f"Deal {deal_id} updated")
    except ApiException as deal:
        return print("Exception when updatin deal: %s\n" %deal)  
    

In [149]:
# Résultat
deal_id="7280660186"
amount=25000
update_deal(deal_id,amount)

Deal 7280660186 updated


# # Calcul du nombre de contacts dont l'attribut En poste = Oui

In [150]:
def count_en_poste(etat="Oui"):

    # Récupération des id des contacts
    contacts_id=[]
    contacts=api_client.crm.contacts.get_all()
    for contact in contacts:
        contacts_id.append(contact.to_dict()["id"])

    # Calcul du nombre de contact selon l'état de l'attribut en poste
    cpt_en_poste=0
    for id in contacts_id:
        contact_data=api_client.crm.contacts.basic_api.get_by_id(id,properties=["en_poste"]).to_dict()
        if contact_data["properties"]["en_poste"] == etat:
            cpt_en_poste+=1
    return cpt_en_poste

In [151]:
# Résultat
count_en_poste()

7

# Nom et prénom du propriétaire actuel

In [152]:
def get_owner():
    owner_id=api_client.crm.companies.basic_api.get_by_id(7496989169,properties=["hubspot_owner_id"]).to_dict()
    owner_id=owner_id["properties"]["hubspot_owner_id"]
    owner_data=api_client.crm.owners.owners_api.get_by_id(owner_id).to_dict()
    return [owner_data["first_name"],owner_data["last_name"]]

In [153]:
# Résultat
get_owner()

['Abd', 'TDG']

# Mise à jour des infos de l'entreprise

In [154]:
def update_company(comp_id="7496989169"):
        try:
                update_montant_tot_deals = SimplePublicObjectInput(properties={
                "valeur_totale_des_deals":total_deal_value() ,
                "valeur_totale_des_deals_ponderee":total_deal_value_weighted(),
                "nombre_de_points_de_contacts_en_poste": count_en_poste(),
                "nom_du_dirigeant":get_owner()[1],
                "prenom_du_dirigeant":get_owner()[0]})
                api_client.crm.companies.basic_api.update(simple_public_object_input=update_montant_tot_deals,company_id=comp_id)
                return print("company updated")
        except ApiException as update:
                return print("Exception when updating company: %s\n" % update)
        

In [155]:
# Résultat
update_company()

company updated


# Récupération des infos de l'entreprise

In [130]:
def comp_info(comp_id="7496989169"):
    company_data=api_client.crm.companies.basic_api.get_by_id(comp_id,properties=["name","city","country","industry",
                                                                             "nom_du_dirigeant","prenom_du_dirigeant",
                                                                             "numberofemployees","valeur_totale_des_deals",
                                                                             "valeur_totale_des_deals_ponderee","num_associated_deals",
                                                                             "nombre_de_points_de_contacts_en_poste",
                                                                             "nombre_de_points_de_contacts_en_poste"]).to_dict()
    return company_data["properties"]

In [156]:
# Résultat
comp_info()

{'city': 'Grenoble',
 'country': 'France',
 'createdate': '2023-04-28T09:53:40.624Z',
 'hs_lastmodifieddate': '2023-05-01T00:17:15.484Z',
 'hs_object_id': '7496989169',
 'industry': 'INFORMATION_TECHNOLOGY_AND_SERVICES',
 'name': 'ABD Labs',
 'nom_du_dirigeant': 'TDG',
 'nombre_de_points_de_contacts_en_poste': '7',
 'num_associated_deals': '3',
 'numberofemployees': '20',
 'prenom_du_dirigeant': 'Abd',
 'valeur_totale_des_deals': '690098.0',
 'valeur_totale_des_deals_ponderee': '318588.2'}